In [ ]:
# Importing the libraries we will need 
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
tf.__version__

'2.3.0'

In [ ]:
# Import the dataset 
data = pd.read_csv("Churn_Modelling.csv")

# X is the matrix, we use every row (:) and then the columns AFTER the 3rd column and all but the last hence data.iloc[:,3:-1]
X = data.iloc[:,3:-1].values

# y is the dependent variable or the variable we are predicting and we import every row (:), and we only import the last column hence [:,-1]
y = data.iloc[:,-1].values

In [ ]:
print(X)
print(y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[1 0 1 ... 1 1 0]


In [ ]:
# We have a few cagtegorical variables, so we need to encode them (country and gender), we also need to import a few more things
from sklearn.preprocessing import LabelEncoder

# Encoding gender
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [ ]:
# Depending on the data we have, we might run into situations where, after label encoding, we might confuse our model into thinking that a column has data with
# some kind of order or hierarchy when we clearly don’t have it. To avoid this, we ‘OneHotEncode’ that column. What one hot encoding does is, it takes a column
# which has categorical data, which has been label encoded and then splits the column into multiple columns. The numbers are replaced by 1s and 0s, 
# depending on which column has what value. In our example, we’ll get four new columns, one for each country — Japan, U.S, India, and China.

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer([('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [ ]:
# Split the data
from sklearn.model_selection import train_test_split
X_train , X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
# Scale the train and test set 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [ ]:
# Sequential class allows us to build out an ANN in a sequential matter adding layers one after the other from the input all the way to the output
# Remember keras was kinda imported into tf so we call it from the tf library
ann = tf.keras.models.Sequential()

In [ ]:
# We will use the dense class in order to add our first input and hidden layers
ann.add(tf.keras.layers.Dense(units = 6, activation= 'relu'))

# We use the same line of code to add another hidden layer
ann.add(tf.keras.layers.Dense(units = 6, activation= 'relu'))

In [ ]:
# Now we need to add the output layer to the model we do almost the same thing, but we change the number of units and the activation function
# The number of units is one because we are predicting a binary variable
# We don't want to have the rectifier fxn (relu) to be predicting the dependent var so we change to sigmoid bc it gives us the probability that a customer decides
# to leave the bank (gives us the probability of a certain outcome) sigmoid is probably the most common fxn used for output layers that are binary
ann.add(tf.keras.layers.Dense(units = 1, activation= 'sigmoid'))

In [ ]:
# Now we have to compile the ANN, the parameter for the compile method are the optimizer, the loss fxn, and the metric
# Optimizers that can perform Stochastic Gradient Descent (SGD) is usually the best performing so we use adam
# Loss fxn for binary outcomes will always be binary_crossentropy otherwise if the outcome is not binary we use categorical_crossentropy
# Also, if we have more than 2 categories, the activation fxn for the output layer should be softmax
ann.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics= ['accuracy'])

In [ ]:
# Now we need to train the model, the batch size is more efficient and has better performance when training an ANN (Batch Learning), it's not comparing results one 
# by one, but it's doing it for a whole batch instead and the classic batch size is 32 so we use 32, but it is a hyperparameter so we can tune it for better 
# performance if needed. The last parameter is epochs (I like to call it a runthrough), which is the number of times we essentially train the model
# (don't use small # for epochs bc it wont be as accurate)

ann.fit(X_train, y_train, batch_size=32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5696 - accuracy: 0.7440
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4597 - accuracy: 0.7972
Epoch 3/100
250/250 [==============================] - 0s 984us/step - loss: 0.4359 - accuracy: 0.8065
Epoch 4/100
250/250 [==============================] - 0s 947us/step - loss: 0.4252 - accuracy: 0.8127
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4160 - accuracy: 0.8211
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4054 - accuracy: 0.8285
Epoch 7/100
250/250 [==============================] - 0s 946us/step - loss: 0.3933 - accuracy: 0.8385
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3806 - accuracy: 0.8445
Epoch 9/100
250/250 [==============================] - 0s 960us/step - loss: 0.3697 - accuracy: 0.8486
Epoch 10/100
250/250 [==============================] - 0s 937us/step - loss: 0.362

In [ ]:
# Now we need to call the predict method to predict a certain customer's churn whether or not they leave the bank (the predict method takes double sq brackets [[]])

ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]])))
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]])) > 0.5)


[[0.0208101]]
[[False]]


In [ ]:
# Now we need to predict the test data to see how the model performs. 
y_pred = ann.predict(X_test)

# Makes it 1 if greater than 0.5
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)), 1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [ ]:
# Create a confusion matrix to visualize the num of TP, TN, FP, FN
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[1526   69]
 [ 204  201]]


0.8635